In [8]:
import openrouteservice as ors
import folium
import json
from openrouteservice import client 
from itertools import permutations

#"mapcoordinates" : [[16.373819, 48.208176], [15.439504, 47.070713]],
#"endcoordinates": [11.811360, 47.404620] 
#orsv2 = client.Client(key='5b3ce3597851110001cf6248bfa98be967114dca935030db643d7c13')
orsv2 = client.Client(base_url='http://localhost:8080/ors')    
    

with open("hkauto2karte.json", "r") as f: 
    data = json.load(f)

#full the list of coordinates with all of the required points needed for the trip
contractcoords = data["mapcoordinates"]
coords = []
for point in contractcoords:
    coords.append(point)
coords.append(data["endcoordinates"])



route_request1 = {'coordinates': coords,
                     'format_out': 'geojson',
                     'profile': 'driving-car',
                     'preference': 'fastest'
                     }

route = orsv2.directions(**route_request1)




#open a json file and load it into var data

#create the map with the given start coordinates (priority is starting in wiesing)
m = folium.Map(location=list(reversed(data["startcoordinates"])), tiles="cartodbpositron")

for marker in coords:
    folium.Marker(location=list(reversed(marker))).add_to(m)

folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']], color='blue').add_to(m)

m

In [ ]:
import openrouteservice
import json
from itertools import permutations


#additional test coords: [15.6229118, 48.2043985,1,1,3],[13.0464806,47.7981346,1,1,2]

def calculate_travel_time(client, coordinates):
    # Send route request
    response = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson'
    )
    # Extract travel time
    travel_time = response['features'][0]['properties']['segments'][0]['duration']
    return travel_time

# Main function to calculate routes
def calculate_routes(start_coordinates, coordinates_list):
    # Initialize OpenRouteService client
    client = openrouteservice.Client(key='5b3ce3597851110001cf6248bfa98be967114dca935030db643d7c13')

    # Generate all possible permutations of coordinates
    all_permutations = permutations(coordinates_list)

    # Initialize result list
    valid_routes = []

    # Add start coordinates
    current_route = [start_coordinates]

    # Iterate through all possible routes
    for permutation in all_permutations:
        total_travel_time = 0
        current_coordinates = start_coordinates
        current_vehicle_type = None
        current_personal_required = None
        route_segments = []

        for coordinate in permutation:
            # Extract latitude and longitude from coordinate
            lat, lon, contract_time, contract_vehicle_type, personal_required, *rest = coordinate

            #Check if the vehicle types match or the required personal match
            if current_vehicle_type is None or personal_required is None:
                current_vehicle_type = contract_vehicle_type
                current_personal_required = personal_required
            elif current_vehicle_type != contract_vehicle_type or current_personal_required != personal_required:
                break
            
           
           # Calculate travel time to the next coordinate pair
            travel_time = calculate_travel_time(client, [current_coordinates[:2], [lat, lon]])

            # Add contract time to total travel time
            total_travel_time += travel_time + (contract_time * 60)  # Convert contract time from minutes to seconds

            # Check if cumulative travel time is under 10 hours
            if total_travel_time <= 10 * 3600:  # Convert 10 hours to seconds
                current_route.append([lat, lon])
                current_coordinates = [lat, lon]
            else:
                break

        # Check if the route is valid and add to the result list
        if total_travel_time <= 10 * 3600:
            valid_routes.append((current_route.copy(), total_travel_time))

        # Reset route for the next permutation
        current_route = [start_coordinates]

    return valid_routes


# Read the JSON file
with open("test1.json", "r") as f: 
    data = json.load(f)
    
# Extract start coordinates and list of coordinate pairs
start_coordinates = data["startcoordinates"]
coordinates_list = data["contractcoordinates"]

# Calculate valid routes
valid_routes = calculate_routes(start_coordinates, coordinates_list)

# Output valid routes and their travel times

for route, travel_time in valid_routes:
    print(f"Route: {route}, Total Travel Time: {travel_time / 3600:.2f} hours")





Route: [[8.641765, 49.398628], [8.683665, 49.419231], [8.665329, 49.414687]], Total Travel Time: 1.55 hours
Route: [[8.641765, 49.398628], [8.665329, 49.414687], [8.683665, 49.419231]], Total Travel Time: 1.58 hours


In [ ]:
import openrouteservice
import json
import math
from itertools import permutations


def calculate_travel_time(client, coordinates):
    # Send route request
    response = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson'
    )
    # Extract travel time
    travel_time = response['features'][0]['properties']['segments'][0]['duration']
    return travel_time


# Modify the main function to use the split_routes function
def calculate_routes(start_coordinates, coordinates_list):
    client = openrouteservice.Client(key='5b3ce3597851110001cf6248bfa98be967114dca935030db643d7c13')
    all_permutations = permutations(coordinates_list)
    valid_routes = []
    

    for permutation in all_permutations:
        total_travel_time = 0
        current_route = [start_coordinates]
        daily_route = []
        current_coordinates = start_coordinates
        current_vehicle_type = None
        current_personal_required = None
        

        for coordinate in permutation:
            lat, lon, contract_time, contract_vehicle_type, personal_required, location_name, *rest = coordinate
            

            if current_vehicle_type is None or personal_required is None:
                current_vehicle_type = contract_vehicle_type
                current_personal_required = personal_required
            elif current_vehicle_type != contract_vehicle_type or current_personal_required != personal_required:
                break
            
            travel_time = calculate_travel_time(client, [current_coordinates[:2], [lat, lon]]) + (contract_time * 60)
            

            #check if you are past 8h and need to rest
            if (total_travel_time + travel_time) > 8 * 3600:
                #print("eepy, need to rest")
                daily_route.append((current_route.copy(), total_travel_time))
                total_travel_time = 0
                print("Driver(s) need to rest at ", location_name)
                #split route
                current_route = [current_route[-1]]

            current_route.append([lat, lon])
            current_coordinates = [lat, lon]
            
            total_travel_time += travel_time

            if coordinate == permutation[-1]:
                daily_route.append((current_route.copy(), total_travel_time))


        valid_routes.append((daily_route.copy(), total_travel_time))
        
    
    return valid_routes


# Read the JSON file
with open("test1.json", "r") as f: 
    data = json.load(f)
    
# Extract start coordinates and list of coordinate pairs
start_coordinates = data["startcoordinates"]
coordinates_list = data["contractcoordinates"]

# Calculate valid routes
valid_routes = calculate_routes(start_coordinates, coordinates_list)

# Output valid routes and their travel times

for i, (daily_route, total_travel_time) in enumerate(valid_routes):
    print(f"Route option {i + 1}:")
    for j, (route, travel_time) in enumerate(daily_route):
        print(f"Route number {j + 1} took {travel_time / 3600:.2f} (in hours)")
       

Route option 1:
Route number 1 took 1.55 (in hours)
Route option 2:
Route number 1 took 1.58 (in hours)


In [22]:
import openrouteservice
import json
import math
from itertools import permutations
import requests



def calculate_travel_time(client, coordinates):
    # Send route request
    response = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
       
    )
    # Extract travel time
    travel_time = response['features'][0]['properties']['segments'][0]['duration']
    return travel_time


# Modify the main function to use the split_routes function
def calculate_routes(start_coordinates, coordinates_list):
    
    client = openrouteservice.Client(base_url='http://localhost:8080/ors')    
    
    all_permutations = permutations(coordinates_list)
    valid_routes = []
    
    for permutation in all_permutations:
        total_travel_time = 0
        current_route = [start_coordinates]
        daily_route = []
        current_coordinates = start_coordinates
        current_vehicle_type = None
        current_personal_required = None
        current_location_name = None
        
        for coordinate in permutation:
            lat, lon, contract_time, contract_vehicle_type, personal_required, location_name, *rest = coordinate
            
            if current_vehicle_type is None or personal_required is None:
                current_vehicle_type = contract_vehicle_type
                current_personal_required = personal_required
                current_location_name = location_name
            elif current_vehicle_type != contract_vehicle_type or current_personal_required != personal_required:
                break
            
            travel_time = calculate_travel_time(client, [current_coordinates[:2], [lat, lon]]) + (contract_time * 60)
            
            if (total_travel_time + travel_time) > 8 * 3600:
                daily_route.append((current_route.copy(), total_travel_time))
                total_travel_time = 0
                #print("Driver(s) need to rest at ", current_location_name)
                current_route = [current_route[-1]]
                
            current_route.append([lat, lon])
            current_coordinates = [lat, lon]
            total_travel_time += travel_time
            

        # Calculate travel time from the last point of the permutation back to the starting coordinates        
        return_travel_time = calculate_travel_time(client, [current_coordinates[:2], start_coordinates[:2]])
        
        if (total_travel_time + return_travel_time) > 8 * 3600:
            #print("Route: ", current_route, "Total travel time: " , total_travel_time)
            #print("Driver(s) need to rest at ", location_name)
            
            daily_route.append((current_route.copy(), total_travel_time))
            total_travel_time = 0
            current_route = [current_route[-1]]

        total_travel_time += return_travel_time
        current_route.append(start_coordinates)
        daily_route.append((current_route.copy(), total_travel_time))

        valid_routes.append((daily_route.copy(), total_travel_time))
    
    return valid_routes



# Use the correct json file for running locally
with open("testmain.json", "r") as f: 
    data = json.load(f)
    
# Extract start coordinates and list of coordinate pairs
start_coordinates = data["startcoordinates"]
coordinates_list = data["contractcoordinates"]

# Calculate valid routes
valid_routes = calculate_routes(start_coordinates, coordinates_list)

# Output valid routes and their travel times
for i, (daily_route, total_travel_time) in enumerate(valid_routes):

    print(f"Route option {i + 1}:")
    for j, (route, travel_time) in enumerate(daily_route):
        print(f"Route number {j + 1} took {travel_time / 3600:.2f} (in hours) {route}")

      

Route option 1:
Route number 1 took 1.11 (in hours) [[8.641765, 49.398628], [8.683665, 49.419231], [8.641765, 49.398628]]
Route option 2:
Route number 1 took 5.70 (in hours) [[8.641765, 49.398628], [8.626231, 49.371885], [8.641765, 49.398628]]


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import Flask-CORS
import json
from IPython.display import display, Javascript

app = Flask(__name__)
CORS(app)   

@app.route('/', methods=['POST'])
def receive_json():
    data = request.json
    
    # Write the received JSON data to a file
    with open("data.json", "w") as f:
        json.dump(data, f)
    
    #get data from json
    start_coordinates = data["startcoordinates"]
    coordinates_list = data["contractcoordinates"]
    #calculate_routes()
    valid_routes = calculate_routes(start_coordinates, coordinates_list)
    responedata = jsonify(valid_routes) 
    # Execute the cell containing your Python script
    display(Javascript('IPython.notebook.execute_cell()'))
    
    return responedata #return actual json response of calculate_routes

if __name__ == '__main__':
    app.run(port=8888)  # Choose any available port for your Flask server
